In [1]:
!pip3 -q install --no-cache-dir -U crcmod # ‼️ NOTE

%cd /kaggle/working/TipsOmaly/model/demo_sepehr/big_vision 

import jax
import jax.numpy as jnp
import ml_collections
import numpy as np


/kaggle/working/TipsOmaly/model/demo_sepehr/big_vision


In [2]:
# Pick your hero: (WHEN CHANGING THIS, RERUN IMAGE/TEXT EMBEDDING CELLS)
# Give this cell 1-3mins.

# VARIANT, RES = 'B/32', 256
# VARIANT, RES = 'B/16', 224
# VARIANT, RES = 'B/16', 256
# VARIANT, RES = 'B/16', 384
# VARIANT, RES = 'B/16', 512
# VARIANT, RES, N_PATCHES = 'B/16', 'naflex', 256
# VARIANT, RES = 'L/16', 256
# VARIANT, RES = 'L/16', 384
VARIANT, RES = 'L/16', 512
# VARIANT, RES = 'So400m/16', 256
# VARIANT, RES = 'So400m/16', 384
# VARIANT, RES = 'So400m/16', 512
# VARIANT, RES, N_PATCHES = 'So400m/16', 'naflex', 256
# VARIANT, RES = 'So400m/14', 224
# VARIANT, RES = 'So400m/14', 384
# VARIANT, RES = 'g-opt/16', 256
# VARIANT, RES = 'g-opt/16', 384

CKPT = f'siglip2_{VARIANT.lower().replace("/", "")}_{RES}.npz'
TXTVARIANT, PATCH_SIZE = VARIANT.split('/')
EMBDIM = {'B': 768, 'L': 1024, 'So400m': 1152, 'g-opt': 1536}[TXTVARIANT]
# Note: The g-opt vision encoder is paired with a So400m text encoder
TXTVARIANT = 'So400m' if TXTVARIANT == 'g-opt' else TXTVARIANT
PATCH_SIZE = int(PATCH_SIZE)
VOCAB = 256_000
SEQLEN = 64

ROOT_PATH='/kaggle/working/cpt/'
# It is significantly faster to first copy the checkpoint (30s vs 8m30 for B and 1m vs ??? for L)
!test -f {ROOT_PATH}/{CKPT} || gsutil cp gs://big_vision/siglip2/{CKPT} {ROOT_PATH}
print(f'{ROOT_PATH}/{CKPT} ', f'gs://big_vision/siglip2/{CKPT} ')


import big_vision.models.proj.image_text.two_towers as model_mod

model_cfg = ml_collections.ConfigDict(dict(
    image_model='vit',
    image=dict(
        pool_type='map',
        scan=True,
        variant=VARIANT,
    ),
    text_model='proj.image_text.text_transformer',
    text=dict(
        scan=True,
        variant=TXTVARIANT,
        vocab_size=256_000,
    ),
    out_dim=[None, EMBDIM],
    bias_init=-10,  # without this arg, no "b" param is added
))
model = model_mod.Model(**model_cfg)

# Using `init_params` is slower but will lead to `load` below performing sanity-checks.
# init_params = jax.jit(model.init, backend="cpu")(jax.random.PRNGKey(42), jnp.zeros([1, RES, RES, 3], jnp.float32), jnp.zeros([1, SEQLEN], jnp.int32))['params']
init_params = None  # Faster but bypasses loading sanity-checks.

params = model_mod.load(init_params, f'/tmp/{CKPT}', model_cfg)

/kaggle/working/cpt//siglip2_l16_512.npz  gs://big_vision/siglip2/siglip2_l16_512.npz 


2025-10-18 08:36:49.442010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760776609.465097   25691 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760776609.472026   25691 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import big_vision.pp.builder as pp_builder
import big_vision.pp.ops_general
import big_vision.pp.ops_image
import big_vision.pp.ops_text
import big_vision.pp.proj.image_text.ops_naflex
import big_vision.pp.proj.paligemma.ops
import PIL

images = [PIL.Image.open(fname) for fname in [
    'apple-ipod.jpg',
]]
pp_img = pp_builder.get_preprocess_fn(f'resize({RES})|value_range(-1, 1)')
imgs = np.array([pp_img({'image': np.array(image)})['image'] for image in images])
print('imgs', imgs.shape)
zimg, _, out = model.apply({'params': params}, imgs, None)

print('zimg', zimg.shape)

I0000 00:00:1760776631.924955   25691 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2941 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
INFO:2025-10-18 08:37:12,791:jax._src.xla_bridge:924: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-10-18 08:37:12,793:jax._src.xla_bridge:924: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


imgs (1, 512, 512, 3)
self.pool_type: map
zimg (1, 1024)


In [4]:
for k,v in out.items():
    if type(v) == type(out['img/stem']):
        print(f'{k}: {type(v)}, {v.shape}')
    else:
        print(f'{k}: {type(v)}, --')

img/stem: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 32, 32, 1024)
img/with_posemb: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024, 1024)
img/encoder: <class 'dict'>, --
img/encoded: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024, 1024)
img/head_input: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024)
img/pre_logits_2d: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 32, 32, 1024)
img/pre_logits: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024)
img/norm: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1)
img/normalized: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024)
img/2d_norm: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024, 1)
img/2d_normalized: <class 'jaxlib.xla_extension.ArrayImpl'>, (1, 1024, 1024)
t: <class 'jaxlib.xla_extension.ArrayImpl'>, (1,)
t/parameter: <class 'numpy.ndarray'>, --
b: <class 'numpy.ndarray'>, --


In [ ]:
#@title Tokenize and embed texts

# texts with translations into random languages
texts_dict = {
    'an apple': 'tufaha',  # Swahili
    'a picture of an apple': 'ένα μήλο',  # Greek (Modern)
    'an ipod': 'айпод',  # Russian
    'an apple with a note saying "ipod"': 'apple na nóta ag rá "ipod"',  # Irish
    'a cold drink on a hot day': 'en kold drink på en varm dag', # Danish
    'a hot drink on a cold day': 'een hete drank op een koude dag', # Dutch
    'a photo of a cold drink on a hot day': '炎热天气里一杯冷饮的照片', # Chinese
    'a photo of a hot drink on a cold day': 'foto cangkir panas dina dinten tiis',  # Sundanese
    #
    'a photo of two guys in need of caffeine': 'एक तस्वीर जिसमें दो लोगों को कैफीन की ज़रूरत है',  # Hindi
    'a photo of two guys in need of water': 'o imagine a doi tipi care au nevoie de apă',  # Romanian
    'a photo of the SigLIP authors': 'foto para penulis SigLIP',  # Indonesian
    'a photo of a rock band': 'ロックバンドの写真', # Japanese
    'a photo of researchers at Google Brain': 'foto av forskare på Google Brain', # Swedish
    'a photo of researchers at OpenAI': 'OpenAI:n tutkijoiden kuva',  # Finnish
    #
    'cow': 'sapi',  # Indonesian
    'a cow in a tuxedo': 'une vache en smoking',  # French
    'a cow on the beach': 'פרה על החוף', # Hebrew
    #
    # Croatian:
    'a picture of a laptop with the lockscreen on, a cup of cappucino, salt and pepper grinders. The view through the window reveals lake Zürich and the Alps in the background of the city.': "slika prijenosnog računala sa zaključanim zaslonom, šalica cappuccina, mlinci za sol i papar. Pogled kroz prozor otkriva jezero Zürich i Alpe u pozadini grada.",
}

# ‼️ NOTE: SigLIP 2 models work best with lowercase texts
pp_txt = pp_builder.get_preprocess_fn(f'lower(key="text")|tok(length={SEQLEN}, model="gemma", bos="no", eos="sticky", key="text")')


def embed_texts(texts):
  txts = np.array([pp_txt({'text': text})['text'] for text in texts])
  _, ztxt, out = model.apply({'params': params}, None, txts)
  return txts, ztxt, dict(t=out['t'], b=out['b'])


txts, ztxt, out = embed_texts(texts_dict)
print(txts.shape, ztxt.shape)